# Figure 8

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy
import pandas

import dolfin_mech as dmech

import get_invariants

## Parameters

### Mesh

In [ ]:
cube_params = {"path_and_mesh_name":"Data/generic_mesh.xdmf"}

### Material

In [ ]:
params = {
    "alpha":0.16,     # kPa
    "gamma":0.5,      # [-]
    "c1":1.2,         # kPa
    "c2":0.,          # kPa
    "kappa":1e2,      # kPa
    "eta":1e-5,       # kPa
    "rho_solid":1e-6} # g/mm3
mat_params = {"scaling":"linear", "parameters":params}

### Loading

In [ ]:
pe = -0.5    # kPa
pi = -3.0    # kPa
g  = +9.81e3 # mm/s2

h_lst = [0., -0.005, +0.005, -0.0065, +0.0065]

## Computing strain fields

In [ ]:
invariants = {}

for h in h_lst:

    ### computing the unloaded configuration
    Uref, phisref_computation, dV_expi_ini = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=1,
        cube_params=cube_params,
        mat_params=mat_params,
        inertia_params={"applied":True, "rho_val":1e-8},
        step_params={"dt_min":1e-4, "dt_ini":1},
        load_params={"type":"p_boundary_condition0", "f":g, "P0":float(pe), "H":h},
        res_basename="Fig8-unloaded",
        get_results=1,
        verbose=1)

    phisref_imposed = [numpy.random.uniform(low=0.4, high=0.6) for i in range(len(phisref_computation))]

    ### computing the end-exhalation configuration
    U_exhal, phisexhal, dV_unloaded = dmech.run_RivlinCube_PoroHyperelasticity(
        cube_params=cube_params,
        move_params = {"move":True, "U":Uref},
        porosity_params={"type":"function_xml_from_array", "val":phisref_imposed},
        mat_params=mat_params,
        inertia_params={"applied":True, "rho_val":1e-8},
        step_params={"dt_min":1e-4, "dt_ini":0.125},
        load_params={"type":"p_boundary_condition", "f":g, "P0":float(pe), "H":h},
        res_basename="Fig8-exhalation",
        get_results=1,
        verbose=1)

    ### computing the end-inhalation configuration
    U_inhal, phisinhal, dV_unloaded = dmech.run_RivlinCube_PoroHyperelasticity(
        cube_params=cube_params,
        move_params = {"move":True, "U":Uref},
        porosity_params={"type":"function_xml_from_array", "val":phisref_imposed},
        mat_params=mat_params,
        inertia_params={"applied":True, "rho_val":1e-8},
        step_params={"dt_min":1e-4, "dt_ini":0.125},
        load_params={"type":"p_boundary_condition", "f":g, "P0":float(pi), "H":h},
        res_basename="Fig8-inhalation",
        get_results=1,
        verbose=1)

    mesh = dolfin.Mesh()
    mesh_name = str(cube_params["path_and_mesh_name"])
    dolfin.XDMFFile(mesh_name).read(mesh)
    dolfin.ALE.move(mesh, Uref)

    invariants_mesh = get_invariants.get_invariants(U_exhal=U_exhal, U_inhal=U_inhal, mesh=mesh)
    invariants["h="+str(h)] = invariants_mesh

## Generating plots

### [Hurtado et al.]'s data

In [ ]:
J  = pandas.read_table("Data/hurtado_J.dat" , sep="\s+", usecols=["zone", "mean", "mean+std", "mean-std"]).to_dict("list")
I1 = pandas.read_table("Data/hurtado_I1.dat", sep="\s+", usecols=["zone", "mean", "mean+std", "mean-std"]).to_dict("list")
I2 = pandas.read_table("Data/hurtado_I2.dat", sep="\s+", usecols=["zone", "mean", "mean+std", "mean-std"]).to_dict("list")

### I1

In [ ]:
fig, ax = plt.subplots()
data = [[I1["mean"][i], I1["mean+std"][i], I1["mean-std"][i]] for i in range(len(I1["zone"]))]
ax.boxplot(data)
plt.plot(invariants["h=0.0"]["zone"]    , invariants["h=0.0"]["I1^"]    , color="green" , linestyle="-", marker=".", label="Model w/ gravity, H=0"  )
plt.plot(invariants["h=0.005"]["zone"]  , invariants["h=0.005"]["I1^"]  , color="red"   , linestyle="-", marker=".", label="Model w/ gravity, H=+H1")
plt.plot(invariants["h=-0.005"]["zone"] , invariants["h=-0.005"]["I1^"] , color="blue"  , linestyle="-", marker=".", label="Model w/ gravity, H=-H1")
plt.plot(invariants["h=0.0065"]["zone"] , invariants["h=0.0065"]["I1^"] , color="purple", linestyle="-", marker=".", label="Model w/ gravity, H=+H2")
plt.plot(invariants["h=-0.0065"]["zone"], invariants["h=-0.0065"]["I1^"], color="orange", linestyle="-", marker=".", label="Model w/ gravity, H=-H2")
plt.grid()
plt.legend()
plt.show()

### I2

In [ ]:
fig, ax = plt.subplots()
data = [[I2["mean"][i], I2["mean+std"][i], I2["mean-std"][i]] for i in range(len(I2["zone"]))]
ax.boxplot(data)
plt.plot(invariants["h=0.0"]["zone"]    , invariants["h=0.0"]["I2^"]    , color="green" , linestyle="-", marker=".", label="Model w/ gravity, H=0"  )
plt.plot(invariants["h=0.005"]["zone"]  , invariants["h=0.005"]["I2^"]  , color="red"   , linestyle="-", marker=".", label="Model w/ gravity, H=+H1")
plt.plot(invariants["h=-0.005"]["zone"] , invariants["h=-0.005"]["I2^"] , color="blue"  , linestyle="-", marker=".", label="Model w/ gravity, H=-H1")
plt.plot(invariants["h=0.0065"]["zone"] , invariants["h=0.0065"]["I2^"] , color="purple", linestyle="-", marker=".", label="Model w/ gravity, H=+H2")
plt.plot(invariants["h=-0.0065"]["zone"], invariants["h=-0.0065"]["I2^"], color="orange", linestyle="-", marker=".", label="Model w/ gravity, H=-H2")
plt.grid()
plt.legend()
plt.show()

### J

In [ ]:
fig, ax = plt.subplots()
data = [[J["mean"][i], J["mean+std"][i], J["mean-std"][i]] for i in range(len(J["zone"]))]
ax.boxplot(data)
plt.plot(invariants["h=0.0"]["zone"]    , invariants["h=0.0"]["J^"]    , color="green" , linestyle="-", marker=".", label="Model w/ gravity, H=0"  )
plt.plot(invariants["h=0.005"]["zone"]  , invariants["h=0.005"]["J^"]  , color="red"   , linestyle="-", marker=".", label="Model w/ gravity, H=+H1")
plt.plot(invariants["h=-0.005"]["zone"] , invariants["h=-0.005"]["J^"] , color="blue"  , linestyle="-", marker=".", label="Model w/ gravity, H=-H1")
plt.plot(invariants["h=0.0065"]["zone"] , invariants["h=0.0065"]["J^"] , color="purple", linestyle="-", marker=".", label="Model w/ gravity, H=+H2")
plt.plot(invariants["h=-0.0065"]["zone"], invariants["h=-0.0065"]["J^"], color="orange", linestyle="-", marker=".", label="Model w/ gravity, H=-H2")
plt.grid()
plt.legend()
plt.show()